# NB04: BERTopic — Topic Discovery + LLM Annotation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RJuro/unistra-nlp2026/blob/main/notebooks/NB04_bertopic.ipynb)

---

**Learning Goals**

By the end of this notebook you will be able to:

- **Discover latent topics** in text collections without any labels using BERTopic
- **Configure BERTopic components** — embeddings, UMAP, HDBSCAN, and vectorizer — to control topic quality
- **Use LLMs to name topics** via the Groq API, replacing cryptic keyword lists with human-readable labels
- **Visualize and interpret topic models** with interactive charts, document maps, and hierarchies

**Estimated time:** ~90 minutes

> **GPU recommended** — go to **Runtime → Change runtime type → T4 GPU** for faster embedding of 5,000 documents (~30s on GPU vs ~5 min on CPU).

---

In [ ]:
# ── Setup ──────────────────────────────────────────────────────────────────
# Install dependencies (Colab-friendly)
!pip install bertopic[visualization] sentence-transformers umap-learn hdbscan openai pandas numpy scikit-learn datasets -q

# Core
import json
import re
import os
import time
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Sklearn
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

# Dimensionality reduction & clustering
from umap import UMAP
from hdbscan import HDBSCAN

# Sentence embeddings
from sentence_transformers import SentenceTransformer

# BERTopic
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired

# LLM client (OpenAI-compatible)
from openai import OpenAI

print("All imports successful.")

In [ ]:
# ── GPU Check ─────────────────────────────────────────────────────────────
import torch

if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")
else:
    print("No GPU detected — running on CPU.")
    print("Embedding 5,000 documents will take ~5 minutes instead of ~30 seconds.")
    print("To enable GPU: Runtime → Change runtime type → T4 GPU")

## 1. The Dataset: Moltbook

**Moltbook** is a dataset of ~44K posts from an AI agent social network — think of it as a leaked database from the front page of the agent internet. The posts were generated by LLM-powered agents interacting in a simulated social platform, complete with submolts (like subreddits), upvotes, and comments.

Each post is annotated with:
- **9 content categories** (`topic_label`: A through I)
- **5 toxicity levels** (`toxic_level`: 0–4)

Our goal: **ignore the labels entirely** and see whether BERTopic can rediscover meaningful topic structure from the raw text alone. Then we will use an LLM to give those discovered topics human-readable names.

In [ ]:
# ── Load Moltbook from HuggingFace ──────────────────────────────────────
from datasets import load_dataset

dataset = load_dataset("TrustAIRLab/Moltbook", "posts", split="train")
df = dataset.to_pandas()
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nSample post:")
print(df.iloc[0])

In [ ]:
# ── Flatten the nested 'post' column and extract text ─────────────
# The dataset has a nested 'post' dict with 'title' and 'content' keys
df["title"] = df["post"].apply(lambda x: x.get("title", "") if isinstance(x, dict) else "")
df["content"] = df["post"].apply(lambda x: x.get("content", "") if isinstance(x, dict) else "")

# Fill NaN values and ensure string types (some entries may be missing/NaN)
df["title"] = df["title"].fillna("").astype(str)
df["content"] = df["content"].fillna("").astype(str)
df["text"] = (df["title"].str.strip() + " . " + df["content"].str.strip()).str.strip()

# Drop empty texts
df = df[df["text"].str.len() > 3].reset_index(drop=True)

print(f"Posts with valid text: {len(df)}")
print(f"\nTopic label distribution:\n{df['topic_label'].value_counts()}")
print(f"\nToxic level distribution:\n{df['toxic_level'].value_counts()}")
print(f"\n--- Example text ---")
print(df["text"].iloc[0][:300])

In [ ]:
# ── Subsample for speed ────────────────────────────────────────────
# 5000 posts is enough to find meaningful topics while keeping runtime short
df_sample = df.sample(5000, random_state=42).reset_index(drop=True)
print(f"Working with {len(df_sample)} posts")
print(f"Topic labels in sample: {sorted(df_sample['topic_label'].unique())}")

## 2. Text Preprocessing

We apply minimal cleaning: lowercase, remove URLs, and collapse whitespace. BERTopic relies on semantic embeddings, so aggressive preprocessing (stemming, removing punctuation) can actually hurt by destroying meaning that the embedding model understands.

In [ ]:
# ── Simple text cleaning ───────────────────────────────────────────
def clean_text(text) -> str:
    """Lowercase, remove URLs, and collapse whitespace. Handles NaN/float gracefully."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"https?://\S+", "", text)  # remove URLs
    text = re.sub(r"\s+", " ", text)           # collapse whitespace
    return text.strip()


df_sample["text_clean"] = df_sample["text"].apply(clean_text)

# Drop any rows that ended up empty after cleaning
df_sample = df_sample[df_sample["text_clean"].str.len() > 3].reset_index(drop=True)

# Before / after
print("BEFORE cleaning:")
print(df_sample["text"].iloc[0][:200])
print()
print("AFTER cleaning:")
print(df_sample["text_clean"].iloc[0][:200])
print(f"\nSample size after cleaning: {len(df_sample)}")
print(f"Average text length: {df_sample['text_clean'].str.len().mean():.0f} chars")

## 3. Generating Embeddings

BERTopic needs **dense vector embeddings** to find semantic clusters. We use `all-MiniLM-L6-v2`, a fast sentence-transformer that maps each text to a 384-dimensional vector. Documents that are semantically similar end up close together in this vector space.

We pre-compute embeddings separately so we can reuse them across experiments without re-encoding.

In [ ]:
# ── Generate sentence embeddings ──────────────────────────────────
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = embedding_model.encode(
    df_sample["text_clean"].tolist(),
    show_progress_bar=True,
    batch_size=64
)

print(f"Embeddings shape: {embeddings.shape}")

## 4. Configuring BERTopic Components

BERTopic is a **modular pipeline**. Each step can be configured independently:

1. **Embeddings** → dense vectors from a sentence transformer (already done above)
2. **UMAP** → reduces the 384-dimensional embeddings to 5 dimensions while preserving local structure
3. **HDBSCAN** → finds density-based clusters in the reduced space (no need to specify *k*)
4. **CountVectorizer** → extracts the most representative words per cluster using c-TF-IDF
5. **Representation model** → refines topic keywords (we use KeyBERTInspired for coherent labels)

### Key parameters to understand

| Component | Parameter | Effect |
|---|---|---|
| UMAP | `n_neighbors` | Higher = more global structure, lower = more local detail |
| UMAP | `n_components` | Dimensions for HDBSCAN (5 is a good default) |
| HDBSCAN | `min_cluster_size` | Minimum documents per topic (higher = fewer, larger topics) |
| HDBSCAN | `min_samples` | How conservative clustering is (higher = denser cores, more outliers) |
| CountVectorizer | `stop_words` | Words to exclude — **must include domain terms** that appear everywhere |
| CountVectorizer | `ngram_range` | (1,2) captures both single words and bigrams |

### Domain-specific stopwords

This is a crucial practical insight: in the Moltbook dataset, words like "ai", "agent", "agents" appear in virtually every post. If we do not add them as stopwords, **every topic will look the same** — a wall of "ai_agents_agent_ai". Domain stopwords are terms that are too common *in your specific corpus* to be discriminative.

In [ ]:
# ── Configure BERTopic components ─────────────────────────────────

# Custom stopwords: English defaults + common filler words + DOMAIN-SPECIFIC terms.
# Since this is an AI agent social network, words like "ai", "agent", "agents"
# appear in nearly every post and overwhelm topic keywords if not removed.
stopwords = list(ENGLISH_STOP_WORDS) + [
    # Common filler words
    "just", "like", "really", "think", "know", "want",
    "got", "get", "one", "would", "could", "also",
    "even", "much", "way", "thing", "things", "make",
    "going", "need", "new", "use", "using", "used",
    # Moltbook domain words (appear in almost every post — not discriminative)
    "ai", "agent", "agents", "moltbook", "post", "posts",
    "bot", "bots", "human", "humans", "world", "hello",
    "sub", "submolt", "hackerclaw", "todos",
]

# Vectorizer: unigrams + bigrams, require term in at least 5 docs
vectorizer = CountVectorizer(
    stop_words=stopwords,
    ngram_range=(1, 2),
    min_df=5,             # raised from 3 — filters out very rare terms
)

# UMAP: reduce to 5 dimensions, cosine distance
umap_model = UMAP(
    n_neighbors=15,
    n_components=5,
    metric="cosine",
    random_state=42,
)

# HDBSCAN: larger clusters for more distinct, interpretable topics
hdbscan_model = HDBSCAN(
    min_cluster_size=50,   # raised from 15 — fewer but more coherent topics
    min_samples=10,        # makes cluster cores denser, reduces noise topics
    metric="euclidean",
    cluster_selection_method="eom",
    prediction_data=True,
)

# Representation: KeyBERT-inspired for more coherent topic keywords
representation_model = KeyBERTInspired()

print("Components configured:")
print(f"  Vectorizer:      CountVectorizer(ngram_range=(1,2), min_df=5, +domain stopwords)")
print(f"  UMAP:            n_neighbors=15, n_components=5, metric='cosine'")
print(f"  HDBSCAN:         min_cluster_size=50, min_samples=10, method='eom'")
print(f"  Representation:  KeyBERTInspired")

## 5. Training the Topic Model

Now we assemble the components into a BERTopic model and run `fit_transform`. Since we already pre-computed embeddings, we pass them directly — BERTopic skips the embedding step and goes straight to UMAP + HDBSCAN.

The output is:
- `topics`: a list of topic IDs (one per document). Topic `-1` = outlier (not assigned to any topic).
- `probs`: soft assignment probabilities for each document.

In [ ]:
# ── Build and train BERTopic ───────────────────────────────────────
topic_model = BERTopic(
    embedding_model=embedding_model,
    vectorizer_model=vectorizer,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
    verbose=True,
)

topics, probs = topic_model.fit_transform(
    df_sample["text_clean"].tolist(),
    embeddings=embeddings,
)

print(f"\nNumber of topics found: {len(set(topics)) - 1}")  # -1 for outlier topic
print(f"Outlier documents: {sum(t == -1 for t in topics)} ({sum(t == -1 for t in topics)/len(topics)*100:.1f}%)")
print(f"Assigned documents: {sum(t != -1 for t in topics)} ({sum(t != -1 for t in topics)/len(topics)*100:.1f}%)")

## 6. Exploring Topics

BERTopic provides multiple ways to inspect the discovered topics. Let's start with the **topic info table**, which shows each topic's size and representative keywords.

In [ ]:
# ── Topic info table ──────────────────────────────────────────────
topic_info = topic_model.get_topic_info()
print(f"Total topics (including outliers): {len(topic_info)}")
print()
print(topic_info.head(20).to_string())

In [ ]:
# ── Visualize topics: bar chart of top keywords ─────────────────
topic_model.visualize_barchart(top_n_topics=15, n_words=8)

In [ ]:
# ── Visualize topic map (intertopic distance) ───────────────────
# Each circle is a topic; size = number of documents; distance = similarity
topic_model.visualize_topics()

In [ ]:
# ── Visualize documents in 2D ─────────────────────────────────────
# Each dot is a document, colored by topic assignment
topic_model.visualize_documents(
    df_sample["text_clean"].tolist(),
    embeddings=embeddings,
    hide_annotations=True,
)

In [ ]:
# ── Topic hierarchy (dendrogram) ─────────────────────────────────
# Shows how topics relate to each other and could be merged
topic_model.visualize_hierarchy()

## 7. LLM-Powered Topic Naming

The keyword-based topic names from BERTopic are functional but not always intuitive. BERTopic has a built-in `OpenAI` representation model that can use any OpenAI-compatible API to generate descriptive topic labels.

We point it at **Groq** with `openai/gpt-oss-20b`. The representation model:
1. Collects top keywords and representative documents for each topic
2. Fills them into a prompt template (via `[DOCUMENTS]` and `[KEYWORDS]` placeholders)
3. Sends it to the LLM and uses the response as the topic name

### Avoiding rate limits

Groq's free tier has strict token-per-minute (TPM) limits. Two key parameters help:
- **`doc_length`** — truncate each representative document to N characters (saves tokens!)
- **`delay_in_seconds`** — pause between API calls to stay under TPM limits
- **`nr_docs`** — how many representative documents to send per topic

In [ ]:
# ── Set up Groq client for LLM topic naming ──────────────────────
import openai
from bertopic.representation import OpenAI as OpenAIRepresentation

GROQ_API_KEY = ""  # @param {type:"string"}

# If not set above, try Colab secrets → then environment variable
if not GROQ_API_KEY:
    try:
        from google.colab import userdata
        GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    except (ImportError, Exception):
        GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "")

groq_client = openai.OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1",
)

# Model choice: gpt-oss-20b supports structured outputs and has 8K TPM on free tier
# (vs 6K TPM for llama-3.1-8b which caused the 413 "Request too large" errors)
LLM_MODEL = "openai/gpt-oss-20b"

# Quick connectivity test
resp = groq_client.chat.completions.create(
    model=LLM_MODEL,
    messages=[{"role": "user", "content": "Say 'ready' in one word."}],
    max_tokens=5,
)
print(f"Model: {LLM_MODEL} — {resp.choices[0].message.content}")

In [ ]:
# ── Create the LLM representation model ──────────────────────────
# This uses BERTopic's built-in OpenAI integration.
# [DOCUMENTS] and [KEYWORDS] are placeholders that BERTopic fills in automatically.

llm_prompt = """I will provide you with sample texts and keywords from a topic cluster.
Your task is to create a concise, descriptive name (3-6 words) that captures the topic's essence.

Requirements:
- Use clear, specific language
- Focus on the core theme, not peripheral details
- Use natural phrasing (avoid generic words like "issues" or "topics")

###EXAMPLES###
Topic:
Sample texts from this topic:
- I just started learning Python and I'm confused about lists vs dictionaries.
- My code keeps throwing a 'KeyError' and I can't figure out why.
Keywords: python, code, programming, error, syntax, function, debug
Topic Name: Beginner Programming & Debugging

Topic:
Sample texts from this topic:
- I meal prep every Sunday but by Wednesday I'm tired of the same thing.
- How do you make healthy eating sustainable with a busy schedule?
Keywords: food, healthy, diet, meal, eating, nutrition, cook
Topic Name: Healthy Eating & Meal Planning
###END EXAMPLES###

Topic:
Sample texts from this topic:
[DOCUMENTS]
Keywords: [KEYWORDS]
Output ONLY the topic name. No explanations. No preamble. Just the topic name:"""

# Create BERTopic's OpenAI representation model pointed at Groq
llm_representation = OpenAIRepresentation(
    client=groq_client,
    model=LLM_MODEL,
    prompt=llm_prompt,
    chat=True,
    nr_docs=3,            # Only send 3 representative docs per topic (saves tokens)
    doc_length=150,        # Truncate each doc to 150 chars (saves tokens!)
    delay_in_seconds=2,    # Wait 2s between API calls to respect 8K TPM limit
)

print("LLM representation model configured:")
print(f"  Model:    {LLM_MODEL}")
print(f"  nr_docs:  3 (representative documents per topic)")
print(f"  doc_length: 150 chars (truncated)")
print(f"  delay:    2s between API calls")

In [ ]:
# ── Apply LLM names to existing topic model ──────────────────────
# update_topics() re-runs ONLY the representation step with the LLM,
# keeping all existing topics and assignments unchanged.
# This will make one API call per topic — expect ~20-60 seconds depending on topic count.

print(f"Updating {len(topic_info) - 1} topics with LLM names (this takes a moment)...\n")

topic_model.update_topics(
    df_sample["text_clean"].tolist(),
    representation_model=llm_representation,
)

# Show the updated topic names
updated_info = topic_model.get_topic_info()
for _, row in updated_info.head(15).iterrows():
    print(f"Topic {row['Topic']:3d} ({row['Count']:4d} docs): {row['Name']}")

# Re-visualize with new LLM-generated names
topic_model.visualize_barchart(top_n_topics=10, n_words=8)

## 8. Comparing Topics to Ground Truth

Moltbook posts have ground-truth `topic_label` categories (A-I). Topic modeling is **unsupervised** — it does not know about these labels. But we can check whether the discovered topics align with the known categories using a cross-tabulation.

A strong alignment means BERTopic found meaningful structure. Mismatches might reveal sub-topics within categories or cross-cutting themes that span multiple categories.

In [ ]:
# ── Cross-tabulation: discovered topics vs ground truth ───────────
df_sample["topic"] = topics

if "topic_label" in df_sample.columns:
    # Show cross-tab for top 15 topics (excluding outliers)
    df_assigned = df_sample[df_sample["topic"] != -1].copy()

    # Limit to top 15 topics by size for readability
    top_topics = df_assigned["topic"].value_counts().head(15).index.tolist()
    df_top = df_assigned[df_assigned["topic"].isin(top_topics)]

    ct = pd.crosstab(
        df_top["topic_label"],
        df_top["topic"],
        margins=True,
    )
    print("Cross-tabulation: Ground-truth category (rows) vs BERTopic topic (columns)")
    print()
    print(ct.to_string())
else:
    print("No 'topic_label' column found -- skipping comparison.")
    print("Topic distribution:")
    print(df_sample["topic"].value_counts().head(15))

## 9. Exercise: Tune the Model

BERTopic is sensitive to its hyperparameters. Your task: **change one or more settings and observe the effect on topic quality.**

Suggestions to try:
- **`min_cluster_size`**: try 10, 20, or 50. Smaller values = more fine-grained topics, larger = fewer broader topics.
- **`n_neighbors`**: try 5, 15, or 30. Controls UMAP's balance between local and global structure.
- **`n_components`**: try 3 or 10 instead of 5.
- **Embedding model**: try `"all-mpnet-base-v2"` (larger, more accurate) or `"paraphrase-MiniLM-L3-v2"` (faster, less accurate).
- **`ngram_range`**: try `(1, 3)` for trigrams.

Compare: How many topics are found? How many outliers? Do the topic keywords look more or less coherent?

In [ ]:
# ── YOUR CODE HERE ─────────────────────────────────────────────────
# Step 1: Change one or more parameters below

# umap_model_v2 = UMAP(
#     n_neighbors=??,       # try 5, 15, 30
#     n_components=??,      # try 3, 5, 10
#     metric="cosine",
#     random_state=42,
# )

# hdbscan_model_v2 = HDBSCAN(
#     min_cluster_size=??,  # try 10, 20, 50
#     metric="euclidean",
#     cluster_selection_method="eom",
#     prediction_data=True,
# )

# Step 2: Build and fit a new topic model

# topic_model_v2 = BERTopic(
#     embedding_model=embedding_model,
#     vectorizer_model=vectorizer,
#     umap_model=umap_model_v2,
#     hdbscan_model=hdbscan_model_v2,
#     representation_model=KeyBERTInspired(),
#     verbose=True,
# )

# topics_v2, probs_v2 = topic_model_v2.fit_transform(
#     df_sample["text_clean"].tolist(),
#     embeddings=embeddings,
# )

# Step 3: Compare
# print(f"Number of topics: {len(set(topics_v2)) - 1}")
# print(f"Outliers: {sum(t == -1 for t in topics_v2)}")
# topic_model_v2.get_topic_info().head(15)

## 10. Summary & Takeaways

### What we learned

1. **BERTopic discovers topics without labels.** It combines sentence embeddings, dimensionality reduction (UMAP), and density-based clustering (HDBSCAN) to find groups of semantically similar documents.

2. **LLMs dramatically improve topic naming.** Instead of reading keyword lists like `["climate", "carbon", "emissions", "energy"]`, an LLM can produce labels like "Climate Change & Carbon Policy" — much easier for humans to interpret.

3. **Key parameters to tune:**
   - `min_cluster_size` controls topic granularity (most impactful parameter)
   - `n_neighbors` balances local vs global structure in UMAP
   - The embedding model determines the quality of the semantic space

4. **When to use topic modeling in research:**
   - Exploratory analysis of large text corpora
   - Discovering themes in survey responses, reviews, or social media
   - Content analysis where manual coding is too expensive
   - As a preprocessing step: topic assignments can become features for downstream tasks

### The BERTopic pipeline at a glance

![BERTopic Pipeline](https://raw.githubusercontent.com/RJuro/unistra-nlp2026/main/notebooks/figures/bertopic_pipeline.png)

### Next steps

- Try **topic merging** with `topic_model.merge_topics()` to combine related topics
- Explore **dynamic topic modeling** for temporal analysis
- Use discovered topics as **features in a classifier** (semi-supervised approach)